In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn import preprocessing
from google.colab import files
from sklearn.utils import resample
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
uploaded = files.upload()

Saving archive.zip to archive (1).zip


In [3]:
!unzip /content/archive.zip

Archive:  /content/archive.zip
replace spam.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [4]:
df = pd.read_csv("/content/spam.csv", encoding='latin-1')

In [5]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [6]:
#droping the columns Unnamed 2, 3 & 4 
df = df.drop(columns = ['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'])

In [7]:
df 

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [8]:
label_encoder = preprocessing.LabelEncoder()
df['v1'] = label_encoder.fit_transform(df['v1'])

In [9]:
#converting the words into vectors 
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df["v2"])
y = df["v1"]

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X.toarray(), y, test_size=0.3, random_state=0)

In [11]:
import keras 
import tensorflow as tf

In [12]:
model = keras.Sequential()
model.add(keras.layers.InputLayer(input_shape=(X_train.shape[1],1)))
model.add(keras.layers.LSTM(64))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(256,activation='relu'))
model.add(keras.layers.Dense(64,activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1,activation='sigmoid')) #output layer 

In [13]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                16896     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 256)               16640     
                                                                 
 dense_1 (Dense)             (None, 64)                16448     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 50,049
Trainable params: 50,049
Non-traina

In [15]:
history = model.fit(X_train, y_train,epochs=5)

Epoch 1/5
122/122 [==============================] - 415s 3s/step - loss: 0.4070 - accuracy: 0.8679
Epoch 2/5
122/122 [==============================] - 411s 3s/step - loss: 0.3947 - accuracy: 0.8695
Epoch 3/5
122/122 [==============================] - 410s 3s/step - loss: 0.3915 - accuracy: 0.8695
Epoch 4/5
122/122 [==============================] - 408s 3s/step - loss: 0.3933 - accuracy: 0.8695
Epoch 5/5
122/122 [==============================] - 409s 3s/step - loss: 0.3939 - accuracy: 0.8695


In [16]:
model.save('sms_classifier.h5')

In [18]:
accr = model.evaluate(X_test,y_test)

53/53 [==============================] - 53s 977ms/step - loss: 0.4181 - accuracy: 0.8577
